# - Importação de bibliotecas -

In [ ]:
import os
import pandas as pd
import numpy as np
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler,
    LabelEncoder,
    OrdinalEncoder,
    OneHotEncoder,
)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import pickle
import ast
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer


warnings.filterwarnings("ignore", category=ConvergenceWarning)

def salvar_melhores(df, nome_base="ranking_melhores", pasta="."):
    for modelo in df['Modelo'].unique():
        df_modelo = df[df['Modelo'] == modelo]
        if not df_modelo.empty:
            nome_arquivo = os.path.join(pasta, f"{nome_base}_{modelo}.csv")
            df.to_csv(nome_arquivo, index=False)
            print(f"Arquivo salvo em: {nome_arquivo}")

#### Durante esses testes foi definido que o modelo com melhor desempenho foi o Random Forest, levando em consideiração suas metricas, então ele será o utilizado para o modelo preditivo na matéria de ciencias de dados esse modelo, com os parâmetros especificados pelos testes aqui feitos:

# - Testes -

## 1. Carregando o dataset

In [ ]:
df = pd.read_csv("base_limpa_1.csv")


X = df.iloc[:, :-1]

y = df.iloc[:, -1] #definindo a ultima como target

if y.dtype == "object": #se o target for categórico, faz o label encoder
    y = LabelEncoder().fit_transform(y)

df.head()

## 2. Definindo Encoders e Scalers

#### Usamos os encoders e scalers que aprendemos em sala de aula, com diferentes casos de uso, para testes mais abrangentes

In [ ]:
# Definindo encoders e scalers que vão ser usados nos testes
encoders = {
    "OrdinalEncoder": OrdinalEncoder(), 
    "OneHotEncoder": OneHotEncoder(drop="first", sparse_output=False), 
    "GetDummies": None,
}

scalers = {
    "StandardScaler": StandardScaler(),
    "MinMaxScaler": MinMaxScaler()
}

## 3. Definindo Modelos e Parâmetros

#### **Modelo:** `RandomForestClassifier(class_weight="balanced")`  
**Hiperparâmetro avaliado:**
- `n_estimators`: [100, 300, 500] → número de árvores na floresta.
- `criterion`: ['gini', 'entropy'] → função para medir a qualidade da divisão.
- `max_depth`: [None, 10, 20, 30] → profundidade máxima das árvores (None = crescimento até parar naturalmente).
- `min_samples_split`: [2, 5, 10] → número mínimo de amostras para dividir um nó.
- `min_samples_leaf`: [1, 2, 4] → número mínimo de amostras por folha.
- `max_features`: ['sqrt', 'log2'] → fração de atributos considerados por divisão.
- `bootstrap`: [True, False] → define se o treinamento usa sampling com reposição.
- `class_weight`: [None, 'balanced'] → ajusta pesos das classes conforme frequência (bom para desbalanceamento).

O Random Forest é **robusto a overfitting** devido ao uso de múltiplas árvores e amostragem aleatória.
A variação desses hiperparâmetros controla a complexidade das árvores, o nível de regularização e a estabilidade do modelo.
O uso de `class_weight="balanced"` ou `'balanced'` na varredura permite corrigir desbalanceamentos nas classes,
garantindo que a floresta não favoreça apenas a classe majoritária.


---

#### **Modelo:** `DecisionTreeClassifier(class_weight="balanced")`  
**Hiperparâmetro avaliado:**
- `ccp_alpha`: valores obtidos a partir do caminho de poda de complexidade da árvore  
  (`DecisionTreeClassifier(class_weight="balanced").cost_complexity_pruning_path(X_train, y_train)['ccp_alphas'][:-1]`)  

O `ccp_alpha` controla a poda de complexidade, reduzindo o overfitting ao eliminar ramos pouco relevantes da floresta.  
O uso de `class_weight="balanced"` ajusta automaticamente os pesos das classes, útil para bases desbalanceadas.

Tiramos o ultimo valor, porque ele normalmente retorna uma arvore vazia

---

#### **Modelo:** `SVC()`  
**Hiperparâmetros avaliados:**
- `C`: [0.1, 1, 10, 100] → controla o trade-off entre margem ampla e erros de classificação.  
- `kernel`: ['linear', 'rbf'] → define o tipo de transformação aplicada aos dados.  
- `gamma`: ['scale', 'auto'] → controla a influência dos pontos de treino no kernel.  
- `degree`: [2, 3, 4] → grau do polinômio (quando kernel='poly').  
- `class_weight`: [None, 'balanced'] → corrige desbalanceamento de classes.  
 
Esses parâmetros definem a **forma e flexibilidade do hiperplano de decisão**, buscando o equilíbrio entre generalização e precisão.

---

#### **Modelo:** `KNeighborsClassifier()`  
**Hiperparâmetros avaliados:**
- `n_neighbors`: [1, 3, 5, 7, 9, 11, 13, 15] → número de vizinhos considerados.  
- `weights`: ['uniform', 'distance'] → define se todos os vizinhos têm o mesmo peso ou se a distância influencia.  
- `metric`: ['minkowski', 'manhattan', 'euclidean'] → tipo de distância usada.  
- `p`: [1, 2] → parâmetro da distância Minkowski (1 = Manhattan, 2 = Euclidiana).  
- `algorithm`: ['auto'] → seleção automática do algoritmo de busca.  
  
O KNN depende fortemente da métrica de distância e do número de vizinhos.  
Esses parâmetros influenciam diretamente a **precisão e robustez** do modelo frente a ruídos.

---

#### **Modelo:** `MLPClassifier(max_iter=2000, early_stopping=True)`  
**Hiperparâmetros avaliados:**
- `hidden_layer_sizes`: [(50,), (100,), (50,50), (100,50), (100,100)] → define a arquitetura da rede.  
- `activation`: ['relu', 'tanh'] → função de ativação dos neurônios.  
- `solver`: ['adam', 'sgd'] → algoritmo de otimização.  
- `alpha`: [0.0001, 0.001, 0.01] → regularização L2 para evitar overfitting.  
- `learning_rate`: ['constant', 'adaptive'] → define como a taxa de aprendizado é ajustada.  
- `learning_rate_init`: [0.001, 0.01] → taxa inicial de aprendizado.  

Os hiperparâmetros da MLP definem **a capacidade de aprendizado e generalização** da rede neural.  
O uso de `early_stopping=True` evita overtraining, interrompendo o treino quando o desempenho de validação não melhora.

In [ ]:
# Definindo os modelos e seus hiperparâmetros pro GridSearch

encoder = OrdinalEncoder()
scaler = StandardScaler()

X_encoded = encoder.fit_transform(X)
X_scaled = scaler.fit_transform(X_encoded)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42
)



modelos = {
    "Random Forest": (
        RandomForestClassifier(class_weight="balanced"),
        {
            'n_estimators': [100, 300, 500],
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 'log2'],
            'bootstrap': [True, False],
            'class_weight': [None, 'balanced']
        },
    ),
    "Decision Tree": (
        DecisionTreeClassifier(class_weight="balanced"),
        {
            'ccp_alpha': DecisionTreeClassifier(class_weight="balanced").cost_complexity_pruning_path(X_train, y_train)['ccp_alphas'][:-1]
        },
    ),
    "SVM": (
        SVC(),
        {
            'C': [0.1, 1, 10, 100],
            'kernel': ['linear', 'rbf'],
            'gamma': ['scale', 'auto'],
            'degree': [2, 3, 4],
            'class_weight': [None, 'balanced']
        },
    ),
    "KNN": (
        KNeighborsClassifier(),
        {
            'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
            'weights': ['uniform', 'distance'],
            'metric': ['minkowski', 'manhattan', 'euclidean'],
            'p': [1, 2],
            'algorithm': ['auto']
        },
    ),
    "MLP Neural Net": (
        MLPClassifier(max_iter=2000, early_stopping=True),
        {
            'hidden_layer_sizes': [(50,), (100,), (50,50), (100,50), (100,100)],
            'activation': ['relu', 'tanh'],
            'solver': ['adam', 'sgd'],
            'alpha': [0.0001, 0.001, 0.01],
            'learning_rate': ['constant', 'adaptive'],
            'learning_rate_init': [0.001, 0.01]
        },
    ),
}


### 3.1 Separação por modelo (para rodar um por vez) 

#### Teste do decision tree com outros parametros

In [ ]:
#Decision Tree
modelos = {
    "Decision Tree": (
        DecisionTreeClassifier(class_weight="balanced"),
        {
            "criterion": ["gini", "entropy"],
            "splitter": ["best", "random"],
            "max_depth": [None, 5, 10],
            "min_samples_split": [2, 5],
            "min_samples_leaf": [1, 2],
            "max_features": [None, "sqrt"],
        },
    ),
}

In [ ]:
#Random Forest
modelos = {
    "Random Forest": (
        RandomForestClassifier(class_weight="balanced"),
        {
            'n_estimators': [100, 300, 500],
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 'log2'],
            'bootstrap': [True, False],
            'class_weight': [None, 'balanced']
        },
    ),
}

In [ ]:
encoder = OrdinalEncoder()
scaler = StandardScaler()

X_encoded = encoder.fit_transform(X)
X_scaled = scaler.fit_transform(X_encoded)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42
)

modelos = {
    "Decision Tree": (
        DecisionTreeClassifier(class_weight="balanced"),
        {
            'ccp_alpha': DecisionTreeClassifier(class_weight="balanced").cost_complexity_pruning_path(X_train, y_train)['ccp_alphas'][:-1]
        },
    ),
}

In [ ]:
#SVM
modelos = {
    "SVM": (
        SVC(),
        {
            'C': [0.1, 1, 10, 100],
            'kernel': ['linear', 'rbf'],
            'gamma': ['scale', 'auto'],
            'degree': [2, 3, 4],
            'class_weight': [None, 'balanced']
        },
    ),
}

In [ ]:
#KNN
modelos = {
    "KNN": (
        KNeighborsClassifier(),
        {
            'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
            'weights': ['uniform', 'distance'],
            'metric': ['minkowski', 'manhattan', 'euclidean'],
            'p': [1, 2],
            'algorithm': ['auto']
        },
    ),
}

In [ ]:
#MLP
modelos = {
    "MLP Neural Net": (
        MLPClassifier(max_iter=2000, early_stopping=True),
        {
            'hidden_layer_sizes': [(50,), (100,), (50,50), (100,50), (100,100)],
            'activation': ['relu', 'tanh'],
            'solver': ['adam', 'sgd'],
            'alpha': [0.0001, 0.001, 0.01],
            'learning_rate': ['constant', 'adaptive'],
            'learning_rate_init': [0.001, 0.01]
        },
    ),
}


## 4. Loop de Testes com GridSearchCV

#### Testes com `GridSearchCV`

#### Fazemos o teste do GridSearchCV, para testar todas as combinações de parametros para cada modelo e repetimos esse teste para cada combinação de encoder e scaler, e colocamos os resultados desses testes em uma lista de dicionarios

In [ ]:
resultados = []

for enc_name, encoder in encoders.items():
    if enc_name == "GetDummies":
        X_enc = pd.get_dummies(X, drop_first=True)
    else:
        X_enc = encoder.fit_transform(X)
        if isinstance(X_enc, np.ndarray):
            X_enc = pd.DataFrame(X_enc)

    for sc_name, scaler in scalers.items():
        X_train, X_test, y_train, y_test = train_test_split(
            X_enc, y, test_size=0.3, random_state=42
        )

        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        for model_name, (modelo, param_grid) in modelos.items():
            try:
                grid = GridSearchCV(
                    modelo, param_grid, cv=5, scoring="f1_weighted", n_jobs=-1
                )
                grid.fit(X_train, y_train)
                y_pred = grid.predict(X_test)

                resultados.append({
                    "Encoder": enc_name,
                    "Scaler": sc_name,
                    "Modelo": model_name,
                    "Melhores Params": grid.best_params_,
                    "Accuracy": accuracy_score(y_test, y_pred),
                    "Precision": precision_score(y_test, y_pred, average="weighted"),
                    "Recall": recall_score(y_test, y_pred, average="weighted"),
                    "F1-Score": f1_score(y_test, y_pred, average="weighted"),
                })

            except Exception as e:
                resultados.append({
                    "Encoder": enc_name,
                    "Scaler": sc_name,
                    "Modelo": model_name,
                    "Melhores Params": None,
                    "Accuracy": None,
                    "Precision": None,
                    "Recall": None,
                    "F1-Score": None,
                    "Erro": str(e),
                })

## 5. Ranking Final dos Modelos

In [ ]:
resultados_df = pd.DataFrame(resultados)
melhores = resultados_df.sort_values(by="F1-Score", ascending=False)

print("Ranking final:")
display(melhores)

In [ ]:
salvar_melhores(melhores)


# - Analise dos resultados dos testes -

In [ ]:
csv_files = [f for f in os.listdir(".") if f.endswith(".csv") and f != "base_limpa_1.csv"]

testes = [pd.read_csv(file) for file in csv_files]
df = pd.concat(testes, ignore_index=True)
df = df.sort_values(by="F1-Score", ascending=False)

display(df)

# - Criando pkl para a melhor combinção de cada modelo -

In [ ]:
modelos = {
    "Decision Tree": DecisionTreeClassifier(class_weight="balanced", random_state=42),
    "Random Forest": RandomForestClassifier(class_weight="balanced", random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "KNN": KNeighborsClassifier(),
    "MLP Neural Net": MLPClassifier(max_iter=2000, early_stopping=True, random_state=42),
}

csv_files = [f for f in os.listdir(".") if f.endswith(".csv") and f != "base_limpa_1.csv"]

for file in csv_files:
    df_result = pd.read_csv(file)
    melhor = df_result.sort_values(by="F1-Score", ascending=False).iloc[0]

    model_name = melhor["Modelo"]
    param_grid = ast.literal_eval(melhor["Melhores Params"])
    encoder_name = melhor["Encoder"]
    scaler_name = melhor["Scaler"]

    df_base = pd.read_csv("base_limpa_1.csv")
    X = df_base.iloc[:, :-1].copy()
    y = df_base.iloc[:, -1].copy()

    if y.dtype == "object":
        y = LabelEncoder().fit_transform(y)

    categorical_cols = X.select_dtypes(include='object').columns.tolist()
    numerical_cols = X.select_dtypes(include=np.number).columns.tolist()

    ordinal_encoders = {}
    onehot_encoders = {}
    scaler = None

    if encoder_name == "GetDummies":
        X = pd.get_dummies(X, drop_first=True)

    elif encoder_name == "OrdinalEncoder":
        for col in categorical_cols:
            ordinal = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
            X[col] = ordinal.fit_transform(X[[col]])
            ordinal_encoders[col] = ordinal

    elif encoder_name == "OneHotEncoder":
        for col in categorical_cols:
            onehot = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
            encoded = onehot.fit_transform(X[[col]])
            encoded_df = pd.DataFrame(encoded, columns=[f"{col}_{i}" for i in range(encoded.shape[1])], index=X.index)
            X = X.drop(columns=[col]).join(encoded_df)
            onehot_encoders[col] = (onehot, encoded_df.columns.tolist())

    numerical_cols = X.select_dtypes(include=np.number).columns.tolist()
    if numerical_cols:
        scaler = StandardScaler() if scaler_name == "StandardScaler" else MinMaxScaler()
        X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )

    pipeline = Pipeline([
        ("modelo", modelos[model_name].set_params(**param_grid))
    ])

    pipeline.fit(X_train, y_train)

    dump_obj = {
        "pipeline": pipeline,
        "encoder_name": encoder_name,
        "scaler_name": scaler_name,
        "ordinal_encoders": ordinal_encoders,
        "onehot_encoders": onehot_encoders,
        "X_train_columns": X_train.columns.tolist(),
        "scaler": scaler
    }

    nome_pkl = f"melhor_{model_name.replace(' ', '_')}.pkl"
    with open(nome_pkl, "wb") as f:
        pickle.dump(dump_obj, f)

### Como carregar

In [ ]:
def carregar_pipeline(nome_pkl):
    with open(nome_pkl, "rb") as f:
        obj = pickle.load(f)
    return obj

def prever_com_pipeline(df, obj):
    X = df.iloc[:, :-1].copy()
    y = df.iloc[:, -1].copy()

    encoder_name = obj["encoder_name"]
    pipeline = obj["pipeline"]
    ordinal_encoders = obj.get("ordinal_encoders", {})
    onehot_encoders = obj.get("onehot_encoders", {})
    X_train_cols = obj.get("X_train_columns")
    scaler = obj.get("scaler")

    if encoder_name == "GetDummies":
        X = pd.get_dummies(X, drop_first=True)
        X = X.reindex(columns=X_train_cols, fill_value=0)
    elif encoder_name == "OrdinalEncoder":
        for col, enc in ordinal_encoders.items():
            X[col] = enc.transform(X[[col]])
    elif encoder_name == "OneHotEncoder":
        for col, (oh, cols) in onehot_encoders.items():
            encoded = oh.transform(X[[col]])
            X = X.drop(columns=[col]).join(pd.DataFrame(encoded, columns=cols, index=X.index))

    num_cols = X.select_dtypes(include="number").columns.tolist()
    if scaler is not None and num_cols:
        X[num_cols] = scaler.transform(X[num_cols])


    y_pred = pipeline.predict(X)
    acc = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred, average="weighted")

    return y_pred, acc, f1
    
df_test = pd.read_csv("base_limpa_1.csv")
obj = carregar_pipeline("melhor_Random_Forest.pkl")

y_pred, acc, f1 = prever_com_pipeline(df_test, obj)
print("Accuracy:", acc)
print("F1-Score:", f1)